In [1]:
import sys
sys.path.append('..')
import CAT
import json
import logging
import numpy as np
import pandas as pd

In [2]:
def setuplogger():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(levelname)s %(asctime)s] %(message)s")
    handler.setFormatter(formatter)
    root.addHandler(handler)

In [3]:
setuplogger()

In [4]:
# choose dataset here
dataset = '2009_skill_builder_data_corrected'
# modify config here
config = {
    'learning_rate': 0.008,
    'batch_size': 64,
    'num_epochs': 10,
    'num_dim': 1, # for IRT or MIRT
    'device': 'cpu',
    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
    'betas': (0.9, 0.999),
}

In [5]:
# read datasets
train_triplets = pd.read_csv(f'../data/{dataset}/train_triples.csv', encoding='utf-8').to_records(index=False)
concept_map = json.load(open(f'../data/{dataset}/concept_map.json', 'r'))
concept_map = {int(k):v for k,v in concept_map.items()}
metadata = json.load(open(f'../data/{dataset}/metadata.json', 'r'))

In [6]:
train_data = CAT.dataset.TrainDataset(train_triplets, concept_map,
                                      metadata['num_train_students'], 
                                      metadata['num_questions'], 
                                      metadata['num_concepts'])

In [7]:
# define model here
model = CAT.model.IRTModel(**config)
# train model
model.init_model(train_data)
model.train(train_data, log_step=10)

[INFO 2024-07-09 13:52:48,817] train on cpu
[INFO 2024-07-09 13:52:48,867] Epoch [1] Batch [0]: loss=inf
[INFO 2024-07-09 13:52:48,876] Epoch [1] Batch [1]: loss=1.38581
[INFO 2024-07-09 13:52:48,884] Epoch [1] Batch [2]: loss=1.03887
[INFO 2024-07-09 13:52:48,892] Epoch [1] Batch [3]: loss=0.92322
[INFO 2024-07-09 13:52:48,899] Epoch [1] Batch [4]: loss=0.86530
[INFO 2024-07-09 13:52:48,906] Epoch [1] Batch [5]: loss=0.83080
[INFO 2024-07-09 13:52:48,914] Epoch [1] Batch [6]: loss=0.80769
[INFO 2024-07-09 13:52:48,922] Epoch [1] Batch [7]: loss=0.79133
[INFO 2024-07-09 13:52:48,930] Epoch [1] Batch [8]: loss=0.77879
[INFO 2024-07-09 13:52:48,937] Epoch [1] Batch [9]: loss=0.76914
[INFO 2024-07-09 13:52:48,945] Epoch [1] Batch [10]: loss=0.76150
[INFO 2024-07-09 13:52:48,951] Epoch [1] Batch [11]: loss=0.75516
[INFO 2024-07-09 13:52:48,958] Epoch [1] Batch [12]: loss=0.74989
[INFO 2024-07-09 13:52:48,965] Epoch [1] Batch [13]: loss=0.74538
[INFO 2024-07-09 13:52:48,972] Epoch [1] Batch

In [8]:
# save model
! mkdir ../ckpt
model.adaptest_save('../ckpt/irt.pt')

mkdir: ../ckpt: File exists
